<a href="https://colab.research.google.com/github/0x0checo/NLP/blob/main/Huggingface_Transformer(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


**Day 3**

*Fine tuning a pretrained model*

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

**Preprocess a dataset**

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets['train']['sentence1'])
tokenized_sentences_2 = tokenizer(raw_datasets['train']['sentence2'])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

token_type_ids, in this example, this is what tells the model which part of the input is the first sentence and which is the second sentence.



In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

这种方法效果很好，但它的缺点是返回一个字典（包含我们的键：`input_ids`、`attention_mask` 和 `token_type_ids`，以及值，这些值是列表的列表）。如果在标记化过程中没有足够的内存来存储整个数据集，它也只会在内存足够的情况下工作（而🤗 Datasets库中的数据集是存储在磁盘上的 Apache Arrow 文件，因此只会将您要求加载到内存中的样本保留在内存中）。

为了将数据保持为数据集，我们将使用 `Dataset.map()` 方法。这还可以为我们提供一些额外的灵活性，如果我们需要执行更多的预处理操作，而不仅仅是标记化。`map()` 方法通过对数据集中的每个元素应用一个函数来工作，因此让我们定义一个函数来标记化我们的输入：

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

这个函数接受一个字典（像我们数据集的条目），并返回一个包含 `input_ids`、`attention_mask` 和 `token_type_ids` 的新字典。请注意，如果示例字典包含多个样本（每个键是一个句子的列表），这个函数仍然有效，因为如前所述，标记器可以处理多个句子对的列表。这将允许我们在调用 `map()` 时使用 `batched=True` 选项，这将大大加快标记化的速度。标记器由🤗 Tokenizers库中的 Rust 编写的标记器支持，这种标记器非常快，但前提是我们一次性提供大量输入。

请注意，我们暂时在标记化函数中没有使用 `padding` 参数。这是因为将所有样本填充到最大长度并不是高效的：最好在构建批次时进行填充，因为那时我们只需要填充到该批次的最大长度，而不是整个数据集的最大长度。当输入的长度差异很大时，这可以节省大量时间和计算资源！

下面是我们如何一次性将标记化函数应用于所有数据集。我们在调用 `map` 时使用 `batched=True`，这样函数就会一次性应用于数据集的多个元素，而不是分别应用于每个元素。这可以加速预处理过程。

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

负责将样本放入批次中的函数叫做 **collate function**（合并函数）。这是您在构建 `DataLoader` 时可以传递的一个参数，默认情况下，合并函数会将样本转换为 PyTorch 张量并将它们拼接在一起（如果您的元素是列表、元组或字典，它会递归地进行拼接）。在我们的情况下，这种方法不可行，因为我们的输入大小并不相同。我们故意推迟了填充操作，直到每个批次需要时才应用填充，从而避免出现填充过多导致输入过长的情况。这将大大加速训练，但请注意，如果您在 TPU 上训练，它可能会引发问题——TPU 更倾向于固定形状的输入，即使这需要额外的填充。

为了实现这一点，我们必须定义一个合并函数，该函数会对我们想要批处理在一起的数据集中的项目应用正确数量的填充。幸运的是，🤗 Transformers 库通过 `DataCollatorWithPadding` 提供了这样的一个函数。在实例化时，它会接收一个标记器（用来确定使用哪种填充符号，以及模型是否期望将填充放在输入的左侧或右侧），并会执行您所需的所有操作：

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

这行代码的作用是创建一个 `DataCollatorWithPadding` 对象，并将其赋值给 `data_collator` 变量。具体解释如下：

- **`DataCollatorWithPadding`**：这是 🤗 Transformers 库提供的一个数据合并器（collator），用于在批处理数据时执行填充（padding）操作。它会自动将输入样本填充到批次内的最大长度。这个合并器通常用于处理输入长度不一致的情况，确保所有样本在批处理时有相同的长度。

- **`tokenizer=tokenizer`**：这里将已经定义的 `tokenizer` 传递给 `DataCollatorWithPadding`。`tokenizer` 用于确定哪些填充符号（padding token）应该被使用，此外，它还会告诉 `DataCollatorWithPadding` 模型是否需要将填充符号放在输入的左侧或右侧。

简而言之，这行代码的作用是设置一个填充合并器，它会在每个批次的样本被打包时根据 `tokenizer` 自动填充样本，以保证每个批次内的样本具有相同的长度。

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

**Fine-tuning a model with the Trainer API**

在我们定义 `Trainer` 之前，第一步是定义一个 `TrainingArguments` 类，它将包含 `Trainer` 在训练和评估过程中使用的所有超参数。您需要提供的唯一参数是一个目录，用于保存训练好的模型以及中间的检查点。对于其他参数，您可以使用默认值，这些默认值应该适用于基本的微调任务。

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
# Define model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

To fine-tune the model on our dataset, we just have to call the train() method of our Trainer:

In [ ]:
trainer.train()

这将开始微调（在 GPU 上应该只需要几分钟），并每 500 步报告一次训练损失。然而，它不会告诉您模型的表现如何（好或坏）。原因如下：

1. 我们没有告诉 `Trainer` 在训练过程中进行评估，未设置 `evaluation_strategy` 为 "steps"（每隔 `eval_steps` 步进行评估）或 "epoch"（在每个 epoch 结束时进行评估）。
2. 我们没有为 `Trainer` 提供一个 `compute_metrics()` 函数来在评估时计算指标（否则，评估只会输出损失，这并不是一个很直观的指标）。

**评估**
让我们看看如何构建一个有用的 `compute_metrics()` 函数，并在下次训练时使用它。这个函数必须接受一个 `EvalPrediction` 对象（这是一个具有 `predictions` 和 `label_ids` 字段的命名元组），并返回一个字典，将字符串映射到浮动数值（字符串是返回的指标名称，浮动数值是它们的值）。为了从我们的模型中获取一些预测，我们可以使用 `Trainer.predict()` 命令：

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

`predict()` 方法的输出是另一个命名元组，包含三个字段：`predictions`、`label_ids` 和 `metrics`。`metrics` 字段将包含传入数据集上的损失值，以及一些时间相关的指标（预测所需的总时间和平均时间）。一旦我们完成了 `compute_metrics()` 函数并将其传递给 `Trainer`，这个字段还会包含 `compute_metrics()` 返回的指标。

如您所见，`predictions` 是一个二维数组，形状为 408 x 2（408 是我们用于预测的数据集中的元素数量）。这些是我们传递给 `predict()` 的每个元素的 logits（正如在上一章中看到的，所有 Transformer 模型都会返回 logits）。为了将它们转换为我们可以与标签进行比较的预测，我们需要在第二个轴上取最大值的索引。

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

现在我们可以将这些预测值（`preds`）与标签进行比较。为了构建我们的 `compute_metric()` 函数，我们将依赖于 🤗 Evaluate 库中的指标。我们可以像加载数据集一样轻松加载与 MRPC 数据集相关的指标，这次使用 `evaluate.load()` 函数。返回的对象具有一个 `compute()` 方法，我们可以用它来进行指标计算：

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

这段代码定义了一个 `compute_metrics` 函数，用于计算模型评估时的性能指标。下面是逐行解释：

1. **`metric = evaluate.load("glue", "mrpc")`**：
   - 这行代码从 🤗 Evaluate 库加载了 MRPC（Microsoft Research Paraphrase Corpus）数据集相关的指标。`"glue"` 是一个包含多种任务的集合，`"mrpc"` 是其中一个任务，专门用于评估句子对是否为同义句（即文本对是否表达相同的意思）。该行代码返回一个可以计算指标的对象。

2. **`logits, labels = eval_preds`**：
   - 这里将 `eval_preds`（一个包含模型预测输出和标签的元组）解包为 `logits` 和 `labels`。`logits` 是模型预测的原始输出值（通常是一个没有经过 softmax 的张量），`labels` 是数据集中真实的标签。

3. **`predictions = np.argmax(logits, axis=-1)`**：
   - 由于 `logits` 是模型的原始输出，它们通常是类别的得分（未归一化的概率）。为了得到最终的预测类别，使用 `np.argmax` 函数沿着最后一个维度（即 `axis=-1`）获取最大值的索引，即选择具有最高得分的类别作为预测结果。

4. **`return metric.compute(predictions=predictions, references=labels)`**：
   - 这行代码使用 `metric` 对象来计算最终的评估指标。`metric.compute` 会根据预测值 (`predictions`) 和真实标签 (`references`，即 `labels`) 计算出相关的指标，如准确率、F1得分等（具体取决于任务）。然后将计算得到的结果返回。

### 总结：
这个函数的作用是：
- 加载MRPC任务的评估指标。
- 解包模型的预测输出（logits）和真实标签（labels）。
- 将 logits 转换为最终的预测类别（通过取最大值的索引）。
- 计算并返回评估指标，如准确率或F1得分等。

In [ ]:
# Final code
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

在深度学习模型中，**model head**（模型头部）是指位于模型的基础网络（如BERT、GPT等预训练模型）之后的部分，通常用于特定任务的输出。这个头部通常由一层或多层额外的神经网络层组成，用来将模型的特征表示（如BERT中的隐层表示）转换为特定任务所需的输出格式。

例如：

- **分类任务**：在文本分类中，模型头部通常是一个全连接层（或多层感知机，MLP），它将BERT模型的输出映射到类别标签的概率分布。
- **序列标注任务**：如命名实体识别（NER）任务，模型头部可能包含多个分类层，每个位置的输出对应一个标签。
- **生成任务**：对于生成任务（如文本生成），模型头部会负责生成下一个词的预测。

在预训练模型中，通常会先有一个基础的模型（如BERT），然后根据目标任务的不同，修改或替换模型的头部。例如，BERT模型默认用于掩码语言建模（Masked Language Modeling, MLM），其头部设计为预测被遮盖的单词。如果你用BERT进行文本分类或其他任务，你可能需要替换这个头部，使用适合该任务的输出层。

在微调过程中，常常会遇到以下情况：
- 如果你使用的是一个已经预训练好的模型（如BERT），而要做的是不同的任务（如文本分类），那么原来的任务头部（例如用于掩码语言建模的头部）会被去掉，并替换为一个新的任务头部（例如用于分类的全连接层）。

**A full training**

在实际编写训练循环之前，我们需要定义一些对象。首先是我们将用于迭代批次的数据加载器（dataloader）。但在我们定义这些数据加载器之前，我们需要对 `tokenized_datasets` 进行一些后处理，以处理一些训练器自动为我们做的事情。具体来说，我们需要：

1. 移除模型不需要的列（如 `sentence1` 和 `sentence2` 列）。
2. 将 `label` 列重命名为 `labels`（因为模型期望这个参数被命名为 `labels`）。
3. 设置数据集的格式，以便它们返回 PyTorch 张量，而不是列表。

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

DataLoader 是 PyTorch 中一个非常重要的类，它用于批量加载数据并将数据分成小批次（batch）。在训练深度学习模型时，我们通常将数据集划分为多个小批次，每次从中读取一个批次的数据进行训练。DataLoader 使得这个过程变得更加方便和高效。

collate_fn=data_collator 这个参数用于指定一个自定义的 collate function（合并函数）来处理数据加载器（DataLoader）在每个批次处理时的行为。

在 PyTorch 中，collate_fn 负责将一个批次的样本组合成一个批次数据（batch）。默认情况下，PyTorch 的 DataLoader 会将数据集中的每个样本按批次加载并自动将它们组合成一个批次，通常是通过简单的拼接操作。然而，某些情况下，尤其是当输入数据的格式或大小不一致时，我们需要自定义如何将这些样本合并为一个批次。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
# Add optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

最后，默认使用的学习率调度器是从最大值（5e-5）到 0 的线性衰减。为了正确地定义它，我们需要知道我们将进行的训练步骤数，这个数值是我们想要运行的 epoch 数量乘以训练批次的数量（即训练数据加载器的长度）。默认情况下，`Trainer` 使用 3 个 epoch，所以我们将按照这个设置：

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

这段代码计算了训练的总步骤数，并为训练设置了一个 线性衰减 的学习率调度器，指定学习率从初始值线性下降到 0。这个调度器将通过 optimizer 来控制学习率的变化，并且不使用 warm-up 步骤。

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

这段代码用于执行模型的训练循环，并在训练过程中显示一个进度条。下面是对代码的逐行解释：

### 1. `from tqdm.auto import tqdm`
- 这行代码从 `tqdm` 库导入了 `tqdm`，它用于显示训练过程中的进度条。`tqdm.auto` 会自动选择合适的进度条格式（例如，Jupyter 笔记本中的进度条会有所不同）。

### 2. `progress_bar = tqdm(range(num_training_steps))`
- 创建一个进度条对象 `progress_bar`，它的总长度是 `num_training_steps`（训练的总步骤数）。`range(num_training_steps)` 生成一个从 0 到 `num_training_steps-1` 的序列，进度条将显示训练的进度。

### 3. `model.train()`
- 将模型设置为训练模式。模型在训练模式下会启用某些特性，例如 Dropout 和 BatchNorm，它们在训练和评估阶段的行为不同。

### 4. `for epoch in range(num_epochs):`
- 这个循环遍历指定数量的训练 epoch（训练次数）。在每次迭代时，整个训练数据集会被使用一次。

### 5. `for batch in train_dataloader:`
- 遍历训练数据加载器（`train_dataloader`）中的每个批次。每个批次包含若干个样本，数据加载器会自动将它们加载到内存中。

### 6. `batch = {k: v.to(device) for k, v in batch.items()}`
- 这行代码将每个批次的数据（`batch`）中的所有张量移动到指定的计算设备（如 GPU 或 CPU）。`device` 是模型运行的设备（可能是 `"cuda"` 或 `"cpu"`）。这确保了模型和数据都在同一个设备上进行计算。

### 7. `outputs = model(**batch)`
- 将批次中的数据传递给模型进行前向计算。`**batch` 使用 Python 的解包语法，将批次字典中的键值对作为模型的输入（例如，`input_ids` 和 `attention_mask`）。
- `outputs` 是模型的输出，通常包括损失值和其他相关数据（例如 logits）。

### 8. `loss = outputs.loss`
- 从模型的输出中提取损失值（`loss`）。在 PyTorch 中，损失是训练过程中用于优化的目标。

### 9. `loss.backward()`
- 执行反向传播（backpropagation），计算梯度。它将根据损失值更新模型中各个参数的梯度。

### 10. `optimizer.step()`
- 执行优化器的 `step()` 操作，更新模型的参数。优化器使用反向传播计算得到的梯度来调整参数，以减少损失。

### 11. `lr_scheduler.step()`
- 调整学习率（如果使用了学习率调度器）。它会根据设定的调度策略（例如线性衰减）更新学习率。

### 12. `optimizer.zero_grad()`
- 清除之前计算的梯度。PyTorch 中的梯度是累积的，因此每次梯度计算之前需要将梯度归零，以防止梯度累积错误。

### 13. `progress_bar.update(1)`
- 更新进度条，每完成一步训练（即一个批次），进度条就前进一步。这会实时显示训练进度，帮助监控训练过程。

### 总结：
这段代码实现了一个标准的训练循环，其中包括：
1. 数据的前向传递。
2. 损失计算和反向传播。
3. 参数更新和学习率调度。
4. 使用进度条显示训练进度。
   
每完成一个批次的数据处理，进度条就会更新一次，从而实时显示训练的进度。

In [ ]:
# Evaluation loop
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()